# Website Demo for Infection Risk Calculator with Information Retrieved from Univeristy Systems

**This is the demo for our website application which estimates infection risk based on building information** 

Import necessary packages and files

In [1]:
from jupyter_dash import JupyterDash
import dash


import os

import sys
import dash_core_components as dcc
import dash_html_components as html
import dash
import sys
import os
sys_path = os.path.dirname(os.getcwd())
#for windows
src_path = sys_path + "\src"
#for mac
#src_path = sys_path + "/src"
sys.path.insert(1, src_path)
from calculator import *
assumptions = var
#for windows
src_path = sys_path + "\notebook"
#for mac
#src_path = sys_path + "/notebook"
sys.path.insert(1, src_path)

In [2]:
#pip install jupyter-dash # Uncomment this to Install jupyter-dash for running HTML in Python

Import data

In [2]:
rid_path = 'rm.csv'
sys_path = os.path.dirname(os.getcwd())


data_path = sys_path + "/data/"

room_df = pd.read_csv(data_path + rid_path)
rooms = room_df.copy()

rooms_id = []
for rid in room_df['Room']:
    rooms_id.append({'label': rid, 'value': rid})
    
rooms.head()

,Room,Area,Height,Windows,Capacity,C19_Occupancy,VAVmin,VAVmax
0,CENTER 101,3428,21.0,NaN,295.0,50.0,3500.0,7000.0
1,CENTER 115,2143,21.0,NaN,196.0,31.0,1290.0,4300.0
2,CENTER 119,2144,21.0,NaN,196.0,30.0,1290.0,4300.0
3,GH 242,2956,NaN,NaN,417.0,50.0,935.0,2325.0
4,LEDDN AUD,1888,NaN,NaN,216.0,30.0,4000.0,12000.0


**Build Website** 

In [5]:
vav_room = {'Select...': [0,0,0]}
index = 0
for i in rooms['Room']:
    vav_room[i] = [rooms['VAVmin'][index], rooms['VAVmax'][index], (rooms['VAVmin'][index] + rooms['VAVmax'][index])/2]
    index += 1
room_names =list(vav_room.keys())

In [18]:
import plotly.express as px

In [86]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
external_scripts = ['event.js']


app = JupyterDash(__name__, external_stylesheets = external_stylesheets, external_scripts = external_scripts)

server = app.server
activities = [{'label':'Lecture', 'value':'Lecture'}, {'label':'Studying', 'value':'Studying'}, {'label':'Singing', 'value':'Singing'}, {'label':'Social', 'value':'Social'}, {'label':'Exercising', 'value':'Exercising'}]




data = pd.DataFrame()
app.layout = html.Div([
    html.H6("Event Information"),
    html.Div(["RoomID: ",
              dcc.Dropdown(id='room-dropdown', value = list(vav_room.keys())[0],options = [{'label':name, 'value':name} for name in room_names])]),
    html.Br(),
    html.Div(["VAV levels: ",
             dcc.Dropdown(id='vav-dropdown')]),
    html.Br(),
    html.Div(["Duration of Event (min): ",
              dcc.Input(id='time-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Number of Occupants: ",
              dcc.Input(id='occupant-input', value = 0, type='number')]),
    html.Br(),
    html.Div(["Activity: ",
              dcc.Dropdown(id='activity-dropdown', value ='test', options=activities)]),
    html.Br(),
    html.Div(["Masks: ",
              dcc.RadioItems(id = 'masks-radio', value = 0,
    options=[
        {'label': 'Masks', 'value': 1},
        {'label': 'No Masks', 'value': 0},
    ], labelStyle={'display': 'inline-block'})  ]),
#     html.Br(),
#     html.Button('Reset', id='reset-button'),
    html.Br(),
    html.Button('Go', id = 'go-button', n_clicks = 0),
    html.Br(),
    html.Div(id = 'calc-output', children = 'Enter values to calculate risk'), 
    html.Button("Add to Visualization", id = "addvi", n_clicks = 0), 
    visdcc.Run_js(id = 'jct'), 
    html.Br(), 
    #html.Div([dcc.Graph(id = 'vi')], style={'width': '70%', 'display': 'inline-block', 'padding': '0 20'})
    html.Div(id = 'return_value')
])

@app.callback(
    dash.dependencies.Output('vav-dropdown', 'options'),
    [dash.dependencies.Input('room-dropdown', 'value')]
)
def update_date_dropdown(name):
    return [{'label': 'cfm min', 'value': "min"}, {'label': 'current', 'value': "current"}, {'label': 'cfm max', 'value': "max"}]

@app.callback(
    dash.dependencies.Output('calc-output', 'children'),
    [dash.dependencies.Input('go-button', 'n_clicks')],
    [dash.dependencies.Input('activity-dropdown', 'value')],
    [dash.dependencies.Input('room-dropdown', 'value')],
    [dash.dependencies.Input('vav-dropdown', 'value')],
    [dash.dependencies.Input('masks-radio', 'value')],
    [dash.dependencies.State('time-input', 'value')],
    [dash.dependencies.State('occupant-input', 'value')]
)
def update_calc(n_clicks, activity_dropdown, room_input,vav_dropdown, mask_tf, time_input, occupant_input):
    if n_clicks >= 1:
        comp_ir = ui_calc(activity_dropdown, room_input, time_input, occupant_input, mask_tf, data_path + rid_path, vav_dropdown)
        total_inf = int(occupant_input * comp_ir)
        to_return = 'The risk of an individual infected because of holding a(n) {} event for {} minutes in {} is {}%, given the most recent infection rates. With {} occupants, it is likely that {} occupant(s) will be infected.'.format(activity_dropdown, 
                                                                                                                                time_input, 
                                                                                                                                room_input, 
                                                                                                                                round((comp_ir * 100),2), 
                                                                                                                                occupant_input,
                                                                                                                                total_inf)
        return to_return
    else:
        return 'Enter Values to get risk calculation' 
   

@app.callback(
    dash.dependencies.Output('return_value', 'children'),
    [dash.dependencies.Input('go-button', 'n_clicks')], 
    [dash.dependencies.Input('activity-dropdown', 'value')],
    [dash.dependencies.Input('room-dropdown', 'value')],
    [dash.dependencies.Input('vav-dropdown', 'value')],
    [dash.dependencies.Input('masks-radio', 'value')],
    [dash.dependencies.State('time-input', 'value')],
    [dash.dependencies.State('occupant-input', 'value')])
def reval(n_clicks, activity_dropdown, room_input,vav_dropdown, mask_tf, time_input, occupant_input):
    if n_clicks >= 1:
        ir = ui_calc(activity_dropdown, room_input, time_input, occupant_input, mask_tf, data_path + rid_path, vav_dropdown)
        results = str({"act": activity_dropdown, "rm": room_input, "ti": time_input, "occupants": occupant_input, "masks": mask_tf, 
                      "ir": round((ir * 100),2)})
        return results
    return ""

@app.callback(
    dash.dependencies.Output('jct', 'run'),
    [dash.dependencies.Input('addvi', 'n_clicks')])
def myfun(n_clicks): 
    if n_clicks >= 1: 
        return "message()"
    return "console.log(0)"

In [87]:
app._terminate_server_for_port("localhost", 8050)
app.run_server()

Dash app running on http://127.0.0.1:8050/
Lecture
lecture
Lecture
The resulting risk of infection is 0.10867451369870329%
It is predicted that 0.001086745136987033 x 10 = 0 susceptible occupants will be infected
lecture
The resulting risk of infection is 2.1266953494145646%
It is predicted that 0.021266953494145646 x 10 = 0 susceptible occupants will be infected
The resulting risk of infection is 0.10867451369870329%
It is predicted that 0.001086745136987033 x 10 = 0 susceptible occupants will be infected
The resulting risk of infection is 2.1266953494145646%
It is predicted that 0.021266953494145646 x 10 = 0 susceptible occupants will be infected
Lecture
Lecturelecture

lecture
The resulting risk of infection is 1.0814459418146405%
It is predicted that 0.010814459418146405 x 100 = 1 susceptible occupants will be infected
The resulting risk of infection is 1.0814459418146405%
The resulting risk of infection is 19.34291479850937%
It is predicted that 0.1934291479850937 x 100 = 19 susce

Lecture
lecture
The resulting risk of infection is 1.0814459418146405%
It is predicted that 0.010814459418146405 x 100 = 1 susceptible occupants will be infected
Lecture
lecture
The resulting risk of infection is 19.34291479850937%
It is predicted that 0.1934291479850937 x 100 = 19 susceptible occupants will be infected
The resulting risk of infection is 1.0814459418146405%
It is predicted that 0.010814459418146405 x 100 = 1 susceptible occupants will be infected
The resulting risk of infection is 19.34291479850937%
It is predicted that 0.1934291479850937 x 100 = 19 susceptible occupants will be infected
Lecture
lecture
Lecture
lecture
The resulting risk of infection is 1.0814459418146405%
It is predicted that 0.010814459418146405 x 100 = 1 susceptible occupants will be infected
The resulting risk of infection is 19.34291479850937%
The resulting risk of infection is 1.0814459418146405%It is predicted that 0.1934291479850937 x 100 = 19 susceptible occupants will be infected

It is predi

Lecture
lectureLecture
lecture

The resulting risk of infection is 0.8835436988337331%The resulting risk of infection is 0.8835436988337331%
It is predicted that 0.00883543698833733 x 100 = 0 susceptible occupants will be infected

It is predicted that 0.00883543698833733 x 100 = 0 susceptible occupants will be infected
The resulting risk of infection is 15.179089079154974%
It is predicted that 0.15179089079154973 x 100 = 15 susceptible occupants will be infected
The resulting risk of infection is 15.179089079154974%
It is predicted that 0.15179089079154973 x 100 = 15 susceptible occupants will be infected
Lecture
lecture
Lecture
lectureThe resulting risk of infection is 0.611636406258631%

It is predicted that 0.00611636406258631 x 100 = 0 susceptible occupants will be infected
The resulting risk of infection is 0.611636406258631%
The resulting risk of infection is 11.42231701895058%
It is predicted that 0.11422317018950578 x 100 = 11 susceptible occupants will be infected
It is predi

In [28]:
!pip install visdcc
import visdcc

  Created wheel for visdcc: filename=visdcc-0.0.40-py3-none-any.whl size=518207 sha256=d49370858d71a83c24f9a9b92445826d0751356847aa01efc8ce9a01a67e6cbf
  Stored in directory: c:\users\hinanawits\appdata\local\pip\cache\wheels\a7\bb\a0\afd0b0c44dea0b07752736845d87f67d1a0ee0f10a8fdc35a5
Successfully built visdcc


In [47]:
?html.Div